In [1]:
from dotenv import load_dotenv

load_dotenv()  # This loads the .env file

import os

import importlib

import agent
import prompt

importlib.reload(agent)
importlib.reload(prompt)

from agent import create_agent
from prompt import build_restaurant_prompt


graph = create_agent()

graph.get_graph().draw_mermaid_png(output_file_path="graph.png")

# Thread-aware config: persist messages per user/thread for memory
thread_config = {
    "configurable": {
        "thread_id": os.getenv("THREAD_ID", "demo-thread"),
        "user_id": os.getenv("USER_ID", "demo-user"),
    }
}

# Build request from prompt template (flexible by location)
request = build_restaurant_prompt(location="Shinjuku, Tokyo, Japan")

INFO:agent:Creating agent
INFO:agent:Memory created
INFO:agent:Graph created


In [2]:
# Stream with cleaner output while capturing inner state
print("=== Streaming Agent Output ===")
final_state = None
for event in graph.stream(request, config=thread_config):
    if "model" in event:
        msg = event["model"]["messages"]
        try:
            content = getattr(msg, "content", None) or str(msg)
        except Exception:
            content = str(msg)
        if content:
            print(content)
    final_state = event

print("\n=== Final Inner State Snapshot ===")
print(final_state)

=== Streaming Agent Output ===


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:tool:GOOGLE_MAPS_API_KEY is set
INFO:tool:session is set
INFO:tool:Results are set
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



=== Final Inner State Snapshot ===
{'agent': {'messages': [AIMessage(content='Here are the top restaurants around Shinjuku, Tokyo, Japan, specializing in Japanese cuisine:\n\n1. **Wagyu Halal Japanese Food**\n   - **Rating:** 4.8\n   - **Reviews Count:** 965\n   - **Price Range:** Not specified\n   - **Types:** Restaurant, Food, Point of Interest, Establishment\n   - **Photo Reference:** AciIO2fncxTOhhHYm-u0Y2K4_1rgYOxMwEzy9uxH2uvdb53flJlNfa00rVKC7SDIb1guC4TOMrWiNwchlh0owfHZIE2tXf4csLLEqWzOYCGmF0UBh6wfJgGtjG0W3Z2QigYCLprtVSVzoXnfsw1EnR_FlK1RuKBxNhdaKp9Fmd8lgKtkm28ZUYijVkpyj6ph6Z21LRruOYsvPrecbZGiINl1fXFTfWBoF_8R003mhm132y3yMMNmnJGstefn7I3porW4wfYo_ssfnZrw_e8W1zelA8PDGkfEgtRRl_Q2Yw4vT91YS4RIM5PlqMhigfz-Tik04ahrxBpWktno_X9t_Q-x3HdoBdMJJmn7z9wpxxBhPQitc0pBvPHBbDUNh9wjCpRROlh5SCZX2R5GKoWmUDD2aflDsG2OXxVC2Vr76To2zS4KRmEsdWDay0Y0sD1B36AWTCxAzg0Iz1Jm6g8w2L3sPN85R4X32ILm3C1n6qZ-le3ao-AhMEIuHiyThunnk1sNmqhOne37T7Vu3goLkThPivInlhyJlIXXID8LN4lqUVz_IcOLc8dvHkqh3Mk9DczABJLdDTiQ4X1qxB0ZFrk2AfMxdRHp

In [3]:
print(final_state["agent"]["messages"][0].content)

Here are the top restaurants around Shinjuku, Tokyo, Japan, specializing in Japanese cuisine:

1. **Wagyu Halal Japanese Food**
   - **Rating:** 4.8
   - **Reviews Count:** 965
   - **Price Range:** Not specified
   - **Types:** Restaurant, Food, Point of Interest, Establishment
   - **Photo Reference:** AciIO2fncxTOhhHYm-u0Y2K4_1rgYOxMwEzy9uxH2uvdb53flJlNfa00rVKC7SDIb1guC4TOMrWiNwchlh0owfHZIE2tXf4csLLEqWzOYCGmF0UBh6wfJgGtjG0W3Z2QigYCLprtVSVzoXnfsw1EnR_FlK1RuKBxNhdaKp9Fmd8lgKtkm28ZUYijVkpyj6ph6Z21LRruOYsvPrecbZGiINl1fXFTfWBoF_8R003mhm132y3yMMNmnJGstefn7I3porW4wfYo_ssfnZrw_e8W1zelA8PDGkfEgtRRl_Q2Yw4vT91YS4RIM5PlqMhigfz-Tik04ahrxBpWktno_X9t_Q-x3HdoBdMJJmn7z9wpxxBhPQitc0pBvPHBbDUNh9wjCpRROlh5SCZX2R5GKoWmUDD2aflDsG2OXxVC2Vr76To2zS4KRmEsdWDay0Y0sD1B36AWTCxAzg0Iz1Jm6g8w2L3sPN85R4X32ILm3C1n6qZ-le3ao-AhMEIuHiyThunnk1sNmqhOne37T7Vu3goLkThPivInlhyJlIXXID8LN4lqUVz_IcOLc8dvHkqh3Mk9DczABJLdDTiQ4X1qxB0ZFrk2AfMxdRHpGg
   - **Place URL:** [Wagyu Halal Japanese Food](https://maps.google.com/?cid=841506

In [4]:
# import importlib
# from dotenv import load_dotenv

# load_dotenv()  # This loads the .env file


# import tool 

# importlib.reload(tool)

# from tool import google_maps_places

# google_maps_places({
#     "query": "sushi",
#     "location": "Shinjuku, Tokyo, Japan",
#     "radius_meters": 2000,
# })
